In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from scipy.spatial import HalfspaceIntersection, ConvexHull
from itertools import combinations
import networkx as nx

## 4.6 Computing correlated equilibria

Mixed strategies in a game correspond to a coin flip / dice roll for each player, but where the probabilities are chosen by the players. At equilibria both players are finding a distribution each that they don't want to deviate from. We assume that the join probability is just the product of the decisions of each agent. So in order to detemine the actions of each agent, we just sample that distribution. However, we can imagine alternatives where both players might be able to agree on a restriction to the joint space. For instance, in the battle of the sexes we have:

$
\begin{array}{c|cc}
\text{} & \text{A} & \text{B} \\
\hline
\text{A} & 2,1 & 0,0 \\
\text{B} & 0,0 & 1,2 \\
\end{array}
$

It is obviously within both player's interest to avoid AB and BA.

Imagine that we had a third party. We both flip our coins with out probabilities, but before doing anything we show the result to the third party, who can tell us if we have hit a spot in the joint distribution we don't want (e.g., HT or TH). Then we just reflip the coins. This enables a greater amount of cooperation!

All we need to do is select joint probabilities such that no party wants to deviate IF all other parties aren't. This can be turned into a linear program, where we are trying to find the joint probabilities. For each possible joint state, we need that for each possible player the benefit of deviation is not greater than 0. For example, taking the above we have 4 joint states, and their utilities:

$
\begin{array}{c|cc}
\text{} & \text{AA} & \text{AB} & \text{BA} & \text{BB} \\
\hline
\text{row player} & 2 & 0 & 0 & 1 \\
\text{column player} & 1 & 0 & 0 & 2 \\
\end{array}
$

We can then get the scores for deviation:

$
\begin{array}{c|cc}
\text{} & \text{AA} & \text{AB} & \text{BA} & \text{BB} \\
\hline
\text{row player deviates to A} & 0 & 0 & 2 & -1 \\
\text{row player deviates to B} & -2 & 1 & 0 & 0 \\
\text{column player deviates to A} & 0 & 1 & 0 & -2 \\
\text{column player deviates to B} & -1 & 0 & 2 & 0 \\
\end{array}
$

We then just need to choose probabilities such that the result is always less than 0. E.g., $[\frac{1}{2},0,0,\frac{1}{2}]$. Under this solution no player wants to deviate.

In [28]:
def get_correlated_equilibrium(U1,U2):
    rows,cols = U1.shape
    deviation_scores = []
    for row in range(rows):
        for col in range(cols):
            # now we know the joint action, and just have to work out the deviation options
            deviation_scores_column = []
            for row_deviation in range(rows):
                deviation_scores_column.append(U1[row_deviation,col]-U1[row,col])
            for col_deviation in range(cols):
                deviation_scores_column.append(U2[row,col_deviation]-U2[row,col])
            deviation_scores.append(deviation_scores_column)

    A = np.array(deviation_scores).T
    b = np.zeros(A.shape[0])
    c = np.random.rand(A.shape[1]) # add some randomness to the objective (which doesn't matter) in order to get different samples
    A_eq = np.ones((1, A.shape[1]))
    b_eq = np.array([1])
    res = linprog(c, A_ub=A, b_ub=b, A_eq=A_eq, b_eq=b_eq)
    print(np.abs(res["x"]).round(3))
    return res["x"]

for i in range(10):
    solution = get_correlated_equilibrium(np.array([[2,0],[0,1]]),np.array([[1,0],[0,2]]))

[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[0.4 0.  0.2 0.4]
[0.4 0.  0.2 0.4]
[0.222 0.444 0.111 0.222]
[0.25 0.5  0.   0.25]


This succeeds at finding a number of different correlated equilibria, though not necessarily the intuitive ones! Every nash equilibria is a correlated equilibria, and so it is not suprising that this finds some of them. Unfortunately, the quick computation of correlated equilibria does not extend to nash equilibria, as with nash equilibria the two agents calculate the probabilities seperately. Thus we cannot have a set of probabilities to fit, without a bunch of other conditions. For instance, the joint probability

$
\begin{array}{c|cc}
\text{} & \text{A} & \text{B} \\
\hline
\text{A} & 1 & 0 \\
\text{B} & 0 & 1 \\
\end{array}
$

is impossible to achieve in the nash situation.